In [21]:
!python -V

Python 3.10.9


In [22]:
import pandas as pd
import boto3
import s3fs

In [23]:
import pickle

In [24]:
import seaborn as sns
import matplotlib.pyplot as plt

In [25]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error

In [26]:
import mlflow
import os


# Fill in with your AWS profile. 
# See https://docs.aws.amazon.com/cli/latest/userguide/cli-configure-files.html#cli-configure-files-using-profiles
os.environ["AWS_PROFILE"] = "" 

# Fill TRACKING_SERVER_HOST with the public DNS of the EC2 instance.
TRACKING_SERVER_HOST = "ec2-52-4-31-201.compute-1.amazonaws.com" 
mlflow.set_tracking_uri(f"http://{TRACKING_SERVER_HOST}:5000")

print(f"tracking URI: '{mlflow.get_tracking_uri()}'")
mlflow.set_experiment("wdc-taxi-ride-fair-prediction-s3")

tracking URI: 'http://ec2-52-4-31-201.compute-1.amazonaws.com:5000'


<Experiment: artifact_location='s3://mlflow-artifacts-remote-ahm-amm/2', creation_time=1690722441341, experiment_id='2', last_update_time=1690722441341, lifecycle_stage='active', name='wdc-taxi-ride-fair-prediction-s3', tags={}>

In [27]:
def read_dataframe(filename):
    
    # Setup AWS S3 
    session = boto3.Session(profile_name='default')
    credentials = session.get_credentials()
    aws_access_key_id = credentials.access_key
    aws_secret_access_key = credentials.secret_key
    region_name = session.region_name

    fs = s3fs.S3FileSystem(
        profile='default',
        key=aws_access_key_id,
        secret=aws_secret_access_key,
        client_kwargs={
        'region_name': region_name
    }
    )

    # Read the TXT file using '|' as the delimiter and specifying the column names
    df = pd.read_csv(filename, delimiter='|', storage_options=dict(profile='default'))

    # Extract the required columns
    df = df[['FAREAMOUNT', 'ORIGIN_BLOCK_LATITUDE', 'ORIGIN_BLOCK_LONGITUDE', 'DESTINATION_BLOCK_LATITUDE', 'DESTINATION_BLOCK_LONGITUDE', 'ORIGINDATETIME_TR']]

    # Convert all headers to lowercase
    df.columns = df.columns.str.lower()
    
    # Drop rows with missing values
    df = df.dropna()
    
    # Print the resulting DataFrame
    print(df.head(1))

    return df

In [28]:
S3_BUCKET_NAME = 'mlflow-artifacts-remote-ahm-amm'
df_train = read_dataframe(f's3://{S3_BUCKET_NAME}/data/train/taxi_2018_01.txt')
df_val = read_dataframe(f's3://{S3_BUCKET_NAME}/data/val/taxi_2019_02.txt')

   fareamount  origin_block_latitude  origin_block_longitude   
0        3.25              38.907734              -77.063516  \

   destination_block_latitude  destination_block_longitude origindatetime_tr  
0                   38.907734                   -77.063516  01/01/2018 04:00  
   fareamount  origin_block_latitude  origin_block_longitude   
0         7.3              38.901486              -77.021916  \

   destination_block_latitude  destination_block_longitude origindatetime_tr  
0                    38.89379                   -77.012491  02/06/2019 16:00  


In [29]:
len(df_train), len(df_val)

(510169, 437856)

In [30]:
def transform_data(df):
    
    # Convert ORIGINDATETIME_TR column to pandas datetime format
    df['origindatetime_tr'] = pd.to_datetime(df['origindatetime_tr'])

    # create new columns for month, day of week, and hour
    df['month_integer'] = df['origindatetime_tr'].dt.month
    df['dow_integer'] = df['origindatetime_tr'].dt.dayofweek
    df['hour_integer'] = df['origindatetime_tr'].dt.hour
    
   # select the final required columns
    df = df[['fareamount', 'origin_block_latitude', 'origin_block_longitude', 'destination_block_latitude', 'destination_block_longitude', 
             'month_integer', 'dow_integer', 'hour_integer']]
    
    return df

In [31]:
df_train = transform_data(df_train)
df_val = transform_data(df_val)
categorical = ['month_integer', 'dow_integer', 'hour_integer']
numerical = ['origin_block_latitude', 'origin_block_longitude', 'destination_block_latitude', 'destination_block_longitude']
dv = DictVectorizer()
train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)
target = 'fareamount'
y_train = df_train[target].values
y_val = df_val[target].values

In [32]:
# Setup AWS S3 
os.environ['AWS_PROFILE'] = 'default'
session = boto3.Session(profile_name='default')
credentials = session.get_credentials()
aws_access_key_id = credentials.access_key
aws_secret_access_key = credentials.secret_key
region_name = session.region_name

fs = s3fs.S3FileSystem(
    profile='default',
    key=aws_access_key_id,
    secret=aws_secret_access_key,
    client_kwargs={
    'region_name': region_name
}
)

with mlflow.start_run():
    
    mlflow.set_tag("developer", "ahmad")
    mlflow.set_tag("model", "lin_reg")
    
    mlflow.log_param("train-data-path", f's3://{S3_BUCKET_NAME}/data/train/taxi_2018_01.txt')
    mlflow.log_param("valid-data-path", f's3://{S3_BUCKET_NAME}/data/val/taxi_2019_02.txt')
    
    lr = LinearRegression()
    lr.fit(X_train, y_train)

    y_pred = lr.predict(X_val)

    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)
    
    with open('models/lin_reg.bin', 'wb') as f_out:
        pickle.dump((dv, lr), f_out)

    mlflow.log_artifact(local_path="models/lin_reg.bin", artifact_path="models_pickle")

In [33]:
with mlflow.start_run():

    mlflow.set_tag("developer", "ahmad")
    mlflow.set_tag("model", "lasso")

    mlflow.log_param("train-data-path", f's3://{S3_BUCKET_NAME}/data/train/taxi_2018_01.txt')
    mlflow.log_param("valid-data-path", f's3://{S3_BUCKET_NAME}/data/val/taxi_2019_02.txt')

    alpha = 0.1
    mlflow.log_param("alpha", alpha)
    lr = Lasso(alpha)
    lr.fit(X_train, y_train)

    y_pred = lr.predict(X_val)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)
    
    with open('models/lasso.bin', 'wb') as f_out:
        pickle.dump((dv, lr), f_out)

    mlflow.log_artifact(local_path="models/lasso.bin", artifact_path="models_pickle")

In [34]:
import xgboost as xgb

In [35]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [36]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [37]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("developer", "ahmad")
        mlflow.set_tag("model", "xgboost_hp")
        mlflow.log_param("train-data-path", f's3://{S3_BUCKET_NAME}/data/train/taxi_2018_01.txt')
        mlflow.log_param("valid-data-path", f's3://{S3_BUCKET_NAME}/data/val/taxi_2019_02.txt')
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50
        )
        y_pred = booster.predict(valid)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

In [38]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=2,
    trials=Trials()
)

[14:00:42] WARNING: ../src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation-rmse:3.04882                          
[1]	validation-rmse:3.35421                          
[2]	validation-rmse:3.55322                          
[3]	validation-rmse:3.65334                          
[4]	validation-rmse:3.67241                          
[5]	validation-rmse:3.68230                          
[6]	validation-rmse:3.68538                          
[7]	validation-rmse:3.70879                          
[8]	validation-rmse:3.73099                          
[9]	validation-rmse:3.75012                          
[10]	validation-rmse:3.79057                         
[11]	validation-rmse:3.81051                         
[12]	validation-rmse:3.84637                         
[13]	validation-rmse:3.86388                         
[14]	validation-rmse:3.89568                         
[15]	validation-rmse:3.90946                         
[16]	validation-r

In [39]:
mlflow.xgboost.autolog(disable=True)

In [40]:
with mlflow.start_run():
    mlflow.set_tag("developer", "ahmad")
    mlflow.set_tag("model", "xgboost_best")
    mlflow.log_param("train-data-path", f's3://{S3_BUCKET_NAME}/data/train/taxi_2018_01.txt')
    mlflow.log_param("valid-data-path", f's3://{S3_BUCKET_NAME}/data/val/taxi_2019_02.txt')
    train = xgb.DMatrix(X_train, label=y_train)
    valid = xgb.DMatrix(X_val, label=y_val)

    best_params = {
        'learning_rate': 0.06616271954179277,
        'max_depth': 93,
        'min_child_weight': 1.3860036417852704,
        'objective': 'reg:linear',
        'reg_alpha': 0.2359754703964827,
        'reg_lambda': 0.01562195969570609,
        'seed': 42
    }

    mlflow.log_params(best_params)

    booster = xgb.train(
        params=best_params,
        dtrain=train,
        num_boost_round=1000,
        evals=[(valid, 'validation')],
        early_stopping_rounds=2
    )

    y_pred = booster.predict(valid)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)

    with open("models/preprocessor.b", "wb") as f_out:
        pickle.dump(dv, f_out)
    mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")
    
    with open('models/xgboost_best.bin', 'wb') as f_out:
        pickle.dump((booster), f_out)

    mlflow.xgboost.log_model(booster, artifact_path="models_mlflow")

[14:06:56] WARNING: ../src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation-rmse:8.79371
[1]	validation-rmse:8.18415
[2]	validation-rmse:7.62279
[3]	validation-rmse:7.10710
[4]	validation-rmse:6.63526
[5]	validation-rmse:6.20314
[6]	validation-rmse:5.81021
[7]	validation-rmse:5.45470
[8]	validation-rmse:5.13491
[9]	validation-rmse:4.84497
[10]	validation-rmse:4.58846
[11]	validation-rmse:4.36063
[12]	validation-rmse:4.16574
[13]	validation-rmse:3.99199
[14]	validation-rmse:3.84131
[15]	validation-rmse:3.71492
[16]	validation-rmse:3.60798
[17]	validation-rmse:3.51935
[18]	validation-rmse:3.44682
[19]	validation-rmse:3.38840
[20]	validation-rmse:3.34429
[21]	validation-rmse:3.31131
[22]	validation-rmse:3.28810
[23]	validation-rmse:3.27328
[24]	validation-rmse:3.26523
[25]	validation-rmse:3.26371
[26]	validation-rmse:3.26910
[27]	validation-rmse:3.27535
